Matrix structure:
```
++++   ++++
+X0+   +X1+
++++   ++++

X0 - books x word emb
X1 - books x cnn emb

E0 - books
E1 - word emb
E2 - cnn emb
```

In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.functional as F

In [2]:
dataset_folder = '../dataset/'
X0_file = dataset_folder + "word2vec_emb_tensor.pkl"
X1_file = dataset_folder + "cnn_emb_tensor.pkl"

In [3]:
X0 = torch.load(X0_file)
print(X0.size())
X1 = torch.load(X1_file)
print(X1.size())

torch.Size([5000, 100])
torch.Size([5000, 1000])


In [4]:
class Autoencoder(nn.Module):
    def __init__(self, input_dim, embedding_dim):
        super().__init__()
        self.enc_linear1 = nn.Linear(input_dim, 128)
        self.enc_linear2 = nn.Linear(128, embedding_dim)
        self.dec_linear1 = nn.Linear(embedding_dim, 128)
        self.dec_linear2 = nn.Linear(128, input_dim)
        self.emb = None
        
    def forward(self, x):
        x = self.enc_linear1(x)
        x = torch.relu(x)
        x = self.enc_linear2(x)
        self.emb = x # return embedding from encoder
        x = torch.relu(x)
        x = self.dec_linear1(x)
        x = torch.relu(x)
        x = self.dec_linear2(x)
        return x # use x for training
    

In [5]:
class matrix_factorization():
    def __init__(self, matrices, entity_list, matrix_entity_mapping, emb_dim):
        self.matrices = matrices
        self.entity_list = entity_list
        self.matrix_entity_mapping = matrix_entity_mapping # {"E0": ["X0", "X1"], "E1": ["X0"], "E2":["X1"]}
        self.emb_dim = emb_dim
        self.autoencoders = {} # {"E0": E0_autoencoder, "E1": E1_ae, ...}
        self.reconstructed_matrices = {} # {"X0": recon_X0, "X1": recon_X1, ...}
        self.embeddings = {} # {"E0": E0_emb, "E1": E1_emb, ...}
        self.concatenated_matrices = []
        self.optim = None
        self.criterion = nn.MSELoss()
        self.batch_size = 50
        self.convergence_threshold = 1e-4
        self.learning_rate = 0.1
        self.epoch_count = 1000
        
    def init_autoencoders(self):
        # initialize autoencoder - one for each entity
        for entity, matrices in matrix_entity_mapping.items():
            if entity == "E0":
                C_E0 = torch.cat((matrices[0], matrices[1]), dim = 1)
                print(C_E0.size())
                E0_aec = Autoencoder(C_E0.size(1), self.emb_dim)
            elif entity == "E1":
                C_E1 = torch.transpose(matrices, 0, 1)
                print(C_E1.size())
                E1_aec = Autoencoder(C_E1.size(1), self.emb_dim)
            elif entity == "E2":
                C_E2 = torch.transpose(matrices, 0, 1)
                print(C_E2.size())
                E2_aec = Autoencoder(C_E2.size(1), self.emb_dim)
                
        self.concatenated_matrices = {"E0": C_E0, "E1": C_E1,"E2": C_E2}
        self.autoencoders = {"E0": E0_aec, "E1": E1_aec, "E2": E2_aec}
        self.optim = torch.optim.SGD(list(E0_aec.parameters()) + list(E1_aec.parameters()) + list(E2_aec.parameters()), lr = self.learning_rate)
    
    def train_autoencoder(self):
        # training
        prev_losses = []
        for epoch in range(0,self.epoch_count):
            shuffled_indices = {}
            avg_loss = {}
            for e in self.autoencoders.keys():
                shuffled_indices[e] = torch.randperm(self.concatenated_matrices[e].size(0))
            
            for e in self.concatenated_matrices.keys():
                total_loss = 0
                num_batches = 0
                for count in range(0, self.concatenated_matrices[e].size(0), self.batch_size):
                    indices = shuffled_indices[e][count:count+self.batch_size]
                    minibatch = self.concatenated_matrices[e][indices]
                    output = self.autoencoders[e](minibatch)
                    loss = self.criterion(minibatch, output)
                    num_batches += 1
                    total_loss += loss
                avg_loss[e] = total_loss/num_batches
    
                
            aec_loss = 0
    
            for v in avg_loss.values():
                aec_loss += v
    
            self.optim.zero_grad()
            aec_loss.requires_grad_(True)
            aec_loss.backward()
            self.optim.step()

            if epoch % 10 == 0:
                print(f"Average loss for epoch {epoch} = {aec_loss}")
            if  (epoch > 100) and (len(prev_losses) > 0) and (prev_losses[-1] - aec_loss < self.convergence_threshold):
                print('Convergence!')
                break
            prev_losses.append(aec_loss)


In [ ]:
matrices = ["X0", "X1"]
entity_list = ["E0", "E1", "E2"]
matrix_entity_mapping = {"E0": (X0, X1), "E1": (X0), "E2":(X1)}
emb_dim = 50

model = matrix_factorization(matrices, entity_list, matrix_entity_mapping, emb_dim)
model.init_autoencoders()
model.train_autoencoder()

torch.Size([5000, 1100])
torch.Size([100, 5000])
torch.Size([1000, 5000])
Average loss for epoch 0 = 3.781221866607666
Average loss for epoch 10 = 3.7677924633026123
Average loss for epoch 20 = 3.75838565826416
Average loss for epoch 30 = 3.747037649154663
Average loss for epoch 40 = 3.729417085647583
Average loss for epoch 50 = 3.6968374252319336
Average loss for epoch 60 = 3.633277416229248
Average loss for epoch 70 = 3.5241165161132812
Average loss for epoch 80 = 3.3786439895629883
Average loss for epoch 90 = 3.2400107383728027
Average loss for epoch 100 = 3.146082878112793
Average loss for epoch 110 = 3.0962650775909424
Average loss for epoch 120 = 3.0727665424346924
Average loss for epoch 130 = 3.061732769012451
Average loss for epoch 140 = 3.0559823513031006
Average loss for epoch 150 = 3.0523102283477783
Average loss for epoch 160 = 3.049363613128662
Average loss for epoch 170 = 3.0465664863586426
Average loss for epoch 180 = 3.0436673164367676
Average loss for epoch 190 = 3.040